**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

In [76]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

def blight_model():
    df1 = pd.read_csv('train.csv',encoding = 'ISO-8859-1')
    # Features I feel are important 
    df1 = df1[['agency_name', 'zip_code', 'violation_code', 'disposition',
           'judgment_amount','compliance']]

    df1.dropna(axis=0,how='any',inplace=True)

#Convert to catergorical codes for better processing
    df1['agency_name'] = pd.Categorical(df1['agency_name'])
    df1['agency_name_d'] = df1['agency_name'].cat.codes + 1

    df1['zip_code'] = pd.Categorical(df1['zip_code'])
    df1['zip_code_d'] = df1['zip_code'].cat.codes + 1 

    df1['violation_code'] = pd.Categorical(df1['violation_code'])
    df1['violation_code_d'] = df1['violation_code'].cat.codes + 1

    df1['disposition'] = pd.Categorical(df1['disposition'])
    df1['disposition_d'] = df1['disposition'].cat.codes + 1
# Keep only the processed columns
    df1 = df1.drop(axis = 1, 
                 labels = ['agency_name','zip_code','violation_code','disposition'])

#Set X_train and y_train
    X_train = df1.drop(axis=1,labels = ['compliance'])
    y_train = df1['compliance']
    
#Define model- Choosen after Grid Search
    model = GradientBoostingClassifier(learning_rate = 0.1 , n_estimators = 100)
    model.fit(X_train,y_train)
    
    
    df = pd.read_csv('test.csv',encoding = 'ISO-8859-1')
    df2 = df[['agency_name', 'zip_code', 'violation_code', 'disposition',
           'judgment_amount']]

    df2['zip_code'] = df['zip_code'].fillna(df['zip_code'].value_counts().index[0])


    df2['agency_name'] = pd.Categorical(df2['agency_name'])
    df2['agency_name_d'] = df2['agency_name'].cat.codes + 1

    df2['zip_code'] = pd.Categorical(df2['zip_code'])
    df2['zip_code_d'] = df2['zip_code'].cat.codes + 1 

    df2['violation_code'] = pd.Categorical(df2['violation_code'])
    df2['violation_code_d'] = df2['violation_code'].cat.codes + 1

    df2['disposition'] = pd.Categorical(df2['disposition'])
    df2['disposition_d'] = df2['disposition'].cat.codes + 1

    df2 = df2.drop(axis = 1, 
                 labels = ['agency_name','zip_code','violation_code','disposition'])
#Test  
    X_test = df2
    y_test = model.predict_proba(X_test)
    answer = pd.DataFrame(data = y_test[:,1], index = df['ticket_id'] )
    return answer

In [68]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying t

,0
ticket_id,
284932,0.408618
285362,0.114999
285361,0.069780
285338,0.222645
285346,0.198364
285345,0.240892
285347,0.161726
285342,0.923445
285530,0.031426


In [70]:
# df = pd.read_csv('test.csv',encoding = 'ISO-8859-1')
# df2 = df[['agency_name', 'zip_code', 'violation_code', 'disposition',
#            'judgment_amount']]

# df2['zip_code'] = df['zip_code'].fillna(df['zip_code'].value_counts().index[0])
# df2.isnull().sum()

In [71]:
# ! cat readonly/test.csv > test.csv